# 1. Lecture CSV

In [503]:
import pandas as pd

import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [504]:
df = pd.read_csv("../datas/Walmart_Store_sales.csv")

In [505]:
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


Store - Numéro du magasin (store number)

Date - Semaine de vente (the week of sales)

Weekly_Sales - Montant de la semaine de vente pour le magasin (sales for the given store)

Holiday_Flag - 1 si la semaine est une semaine de vacances, 0 sinon (whether the week is a special holiday week 1 – Holiday week 0 – Non-holiday week)

Temperature - Temperature le jour de la vente (Temperature on the day of sale)

Fuel_Price - Coût du carburant dans la région (Cost of fuel in the region)

CPI – Indice des prix à la consommation en vigueur (Prevailing consumer price index)

Unemployment - Taux de chômage actuel (Prevailing unemployment rate)

# 2. EDA et préprocessing

In [506]:
# Basic stats
print("Nombre de lignes : {}".format(df.shape[0]))
print()

print("Affichage du dataset : ")
display(df.head())
print()

print("Statistiques basiques : ")
data_desc = df.describe(include="all")
display(data_desc)
print()

print("Valeurs manquantes : ")
display(df.isna().sum())

print("Pourcentage de valeurs manquantes : ")
display(100 * df.isnull().sum() / df.shape[0])

Nombre de lignes : 150

Affichage du dataset : 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092



Statistiques basiques : 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000



Valeurs manquantes : 


Store            0
Date            18
Weekly_Sales    14
Holiday_Flag    12
Temperature     18
Fuel_Price      14
CPI             12
Unemployment    15
dtype: int64

Pourcentage de valeurs manquantes : 


Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

On observe environ 10% de valeurs manquantes pour chaque colonne, sauf `Store`.

------

### Weekly_Sales

Variable cible (Target variable) => `Weekly_Sales`

In [507]:
df["Weekly_Sales"].head()

0    1572117.54
1    1807545.43
2           NaN
3    1244390.03
4    1644470.66
Name: Weekly_Sales, dtype: float64

In [508]:
df["Weekly_Sales"].isna().sum()

np.int64(14)

`Weekly_Sales` contient 14 valeurs vides : étant donné que c'est notre valeur cible, je ne peux pas remplacer les valeurs manquantes donc je les supprime

In [509]:
mask = (~df["Weekly_Sales"].isna())
df = df[mask]
df.shape[0]

136

----

### Date

Gestion de la colonne `Date` : je l'explose en 4 données plus exploitables : l'année, le mois, le jour et le jour de la semaine

In [510]:
df["Date"] = pd.to_datetime(df["Date"], format="%d-%m-%Y")

df["Year"] = df["Date"].dt.year.astype("Int64")
df["Month"] = df["Date"].dt.month.astype("Int64")
df["Day"] = df["Date"].dt.day.astype("Int64")
df["Day_Of_Week"] = df["Date"].dt.dayofweek.astype("Int64")

df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Day_Of_Week
0,6.0,2011-02-18,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011,2,18,4
1,13.0,2011-03-25,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011,3,25,4
3,11.0,NaT,1244390.03,0.0,84.57,NaN,214.556497,7.346,<NA>,<NA>,<NA>,<NA>
4,6.0,2010-05-28,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010,5,28,4
5,4.0,2010-05-28,1857533.70,0.0,NaN,2.756,126.160226,7.896,2010,5,28,4


---

### Type des variables

In [511]:
df.dtypes

Store                  float64
Date            datetime64[ns]
Weekly_Sales           float64
Holiday_Flag           float64
Temperature            float64
Fuel_Price             float64
CPI                    float64
Unemployment           float64
Year                     Int64
Month                    Int64
Day                      Int64
Day_Of_Week              Int64
dtype: object

`Store`

L'identifiant du magasin sera traité comme une variable qualitative

In [512]:
df["Store"].value_counts()

Store
3.0     12
18.0    10
13.0     9
1.0      9
14.0     9
7.0      8
5.0      8
19.0     8
2.0      8
17.0     7
6.0      6
4.0      6
8.0      6
10.0     5
12.0     5
20.0     5
16.0     4
15.0     4
9.0      4
11.0     3
Name: count, dtype: int64

In [513]:
df["Store"] = df["Store"].astype(int).astype(str)

`Holiday_Flag`

In [514]:
# Le flag Vacances est 0 ou 1
df["Holiday_Flag"].value_counts()

Holiday_Flag
0.0    116
1.0      9
Name: count, dtype: int64

Le flag Vacances est une variable qualitative nominale, il n'y a pas d'ordre entre les valeurs

In [515]:
# Il existe des valeurs Null, nous allons les remplacer par la valeur la plus fréquante dans la colonne
mode_holiday = df["Holiday_Flag"].mode()[0]
df["Holiday_Flag"] = df["Holiday_Flag"].fillna(mode_holiday).astype(bool)

In [516]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 136 entries, 0 to 149
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Store         136 non-null    object        
 1   Date          118 non-null    datetime64[ns]
 2   Weekly_Sales  136 non-null    float64       
 3   Holiday_Flag  136 non-null    bool          
 4   Temperature   121 non-null    float64       
 5   Fuel_Price    124 non-null    float64       
 6   CPI           125 non-null    float64       
 7   Unemployment  122 non-null    float64       
 8   Year          118 non-null    Int64         
 9   Month         118 non-null    Int64         
 10  Day           118 non-null    Int64         
 11  Day_Of_Week   118 non-null    Int64         
dtypes: Int64(4), bool(1), datetime64[ns](1), float64(5), object(1)
memory usage: 13.4+ KB


In [517]:
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Day_Of_Week
0,6,2011-02-18,1572117.54,False,59.61,3.045,214.777523,6.858,2011,2,18,4
1,13,2011-03-25,1807545.43,False,42.38,3.435,128.616064,7.470,2011,3,25,4
3,11,NaT,1244390.03,False,84.57,NaN,214.556497,7.346,<NA>,<NA>,<NA>,<NA>
4,6,2010-05-28,1644470.66,False,78.89,2.759,212.412888,7.092,2010,5,28,4
5,4,2010-05-28,1857533.70,False,NaN,2.756,126.160226,7.896,2010,5,28,4


---

### Gestion des valeurs extrêmes

 Pour les colonnes `Temperature`, `Fuel_price`, `CPI` and `Unemployment`. On applique la règle des 3 sigmas : on va détecter les valeurs aberrantes (ou outliers) en considérant que si une valeur se trouve à plus de 3 écarts-types de la moyenne, alors cela peut être un signe qu'elle pourrait être une anomalie.

In [518]:
col_outliers = ["Temperature", "Fuel_Price", "CPI", "Unemployment"]

for col in col_outliers:
    # Calcul des outliers pour les colonnes concernées
    valeur_palier_haut = df[col].mean() + 3 * df[col].std()
    valeur_palier_bas = df[col].mean() - 3 * df[col].std()

    # Application sur le dataframe
    outlier_condition = (df[col] > valeur_palier_haut) | (df[col] < valeur_palier_bas)
    df = df[~outlier_condition]

In [519]:
# Basic stats
print("Nombre de lignes : {}".format(df.shape[0]))
print()

print("Statistiques basiques : ")
data_desc = df.describe(include="all")
display(data_desc)
print()

print("Valeurs manquantes : ")
display(df.isna().sum())

Nombre de lignes : 131

Statistiques basiques : 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Day_Of_Week
count,131,113,1.310000e+02,131,117.000000,119.000000,120.000000,117.000000,113.0,113.0,113.0,113.0
unique,19,NaN,NaN,2,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>
top,3,NaN,NaN,False,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>
freq,12,NaN,NaN,123,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>
mean,NaN,2011-04-24 21:52:33.982300928,1.257990e+06,NaN,60.405897,3.302908,180.175755,7.399427,2010.831858,6.274336,16.530973,4.0
min,NaN,2010-02-05 00:00:00,2.689290e+05,NaN,18.790000,2.514000,126.111903,5.143000,2010.0,1.0,1.0,4.0
25%,NaN,2010-07-30 00:00:00,5.842439e+05,NaN,44.820000,2.824000,132.579257,6.664000,2010.0,4.0,10.0,4.0
50%,NaN,2011-04-22 00:00:00,1.366396e+06,NaN,61.790000,3.435000,197.655672,7.368000,2011.0,6.0,17.0,4.0
75%,NaN,2012-01-13 00:00:00,1.809576e+06,NaN,75.540000,3.708500,214.904838,8.099000,2012.0,9.0,24.0,4.0
max,NaN,2012-10-19 00:00:00,2.771397e+06,NaN,91.650000,4.170000,226.968844,9.524000,2012.0,12.0,31.0,4.0



Valeurs manquantes : 


Store            0
Date            18
Weekly_Sales     0
Holiday_Flag     0
Temperature     14
Fuel_Price      12
CPI             11
Unemployment    14
Year            18
Month           18
Day             18
Day_Of_Week     18
dtype: int64

----

### Distribution des variables

##### Distribution des variables numériques

In [520]:
num_features = df.select_dtypes(include=["float", "int"]).columns

for feature in num_features:
    fig = px.histogram(df, x=feature, color_discrete_sequence=px.colors.qualitative.Pastel, nbins=30)
    fig.update_layout(title=f"Distribution de {feature}", title_x=0.5, xaxis_title=feature, yaxis_title="Nombre d'occurences")
    fig.show()

In [521]:
df["Day_Of_Week"].value_counts()

Day_Of_Week
4    113
Name: count, dtype: Int64

Notre datasat contient des données de ventes hebdomadaires, transmises une fois par semaine, la colonne Day_Of_Week a donc toujours le même jour de la semaine, elle n'est donc pas pertinante.

In [522]:
df = df.drop("Day_Of_Week", axis=1)

##### Répartition des variables qualitatives

In [523]:
cat_features = df.select_dtypes(include=["object", "bool"]).columns

for feature in cat_features:
    df_grouped = df[feature].value_counts().reset_index()
    df_grouped.columns = [feature, "count"]

    fig = px.bar(df_grouped, x=feature, y="count", color_discrete_sequence=px.colors.qualitative.Pastel)
    fig.update_layout(title=f"Répartition de {feature}", title_x=0.5,
        xaxis_title=feature, xaxis=dict(tickformat="d"),
        yaxis_title="Nombre d'occurences")
    fig.show()

---

### Analyse des données

##### Saisonnalité : Nombre de données mensuelles, par année

In [524]:
df["Year"].unique().tolist()

[2011, <NA>, 2010, 2012]

In [525]:
fig = make_subplots(rows = 3, cols = 1)

years = df["Year"].sort_values().dropna().unique().tolist()

for i in range(len(years)):
    mask = (df["Year"] == years[i])
    df_year = df[mask]

    fig.add_trace(go.Histogram(x = df_year["Month"], name = str(years[i]), nbinsx=12), row = i + 1, col = 1)

fig.update_layout(title = go.layout.Title(text = "Données mensuelles par année", x = 0.5), autosize = False, height = 700)

fig.show()

##### Exploration des ventes en fonction du temps

In [526]:
df_sales = df.groupby("Date")["Weekly_Sales"].mean().reset_index()
df_sales["Mean_Sales"] = df_sales["Weekly_Sales"].mean()
df_sales.head()

,Date,Weekly_Sales,Mean_Sales
0,2010-02-05,461622.220,1.255114e+06
1,2010-02-12,1318379.420,1.255114e+06
2,2010-02-19,1392645.145,1.255114e+06
3,2010-02-26,2095591.630,1.255114e+06
4,2010-03-12,860336.160,1.255114e+06


In [527]:
fig = go.Figure(data = 
                go.Scatter(x = df_sales["Date"], y = df_sales["Weekly_Sales"]),
                layout = go.Layout(title = go.layout.Title(text = "Moyenne des ventes par jour", x = 0.5),
                                   xaxis = go.layout.XAxis(title = "Temps", rangeslider = go.layout.xaxis.Rangeslider(visible = True))))

fig.add_trace(go.Scatter(x = df_sales["Date"], y = df_sales["Mean_Sales"]))

fig.update_layout(showlegend = False)

fig.show()

### Gaphique bivarié afin d'analyser la relation de chaque variable avec la cible `Weekly_Sales`

In [528]:
fig = px.scatter_matrix(df)

fig.update_layout(title = go.layout.Title(text = "Bivariate analysis", x = 0.5), showlegend = False, 
            autosize=False, height=1400, width = 1400)

fig.show()

Visuellement, je ne vois pas de corrélation évidente entre une variable du datset et la variable cible. Je vais regarder la matrice de corrélation.

### Matrice de corrélation

In [529]:
# Correlation matrix
corr_matrix = df.corr().round(2)

fig = ff.create_annotated_heatmap(corr_matrix.values, x = corr_matrix.columns.tolist(), y = corr_matrix.index.tolist())
fig.show()

A l'aide de la matrice de corrélation, on observe qu'avec la variable cible `Weekly_Sales`, l'indice de correlation le plus élevé est avec `CPI` puis `Unemployment` puis `Temperature`, puis `Store`.

---

L'EDA étant terminé, la variable `Date`, qui a été explosée en 3 colonnes distinctes `Year`, `Month` et `Day`, peut être supprimée

In [530]:
df = df.drop(columns=["Date"])
df.head()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day
0,6,1572117.54,False,59.61,3.045,214.777523,6.858,2011,2,18
1,13,1807545.43,False,42.38,3.435,128.616064,7.470,2011,3,25
3,11,1244390.03,False,84.57,NaN,214.556497,7.346,<NA>,<NA>,<NA>
4,6,1644470.66,False,78.89,2.759,212.412888,7.092,2010,5,28
5,4,1857533.70,False,NaN,2.756,126.160226,7.896,2010,5,28


---

Sauvegarde du fichier

In [531]:
df.to_csv("../datas/Walmart_Store_sales_clean.csv", index=False)